# Fuzzy matching algorithm

* https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536
* https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import re

In [6]:
!pip install sparse_dot_topn

     |████████████████████████████████| 106 kB 90 kB/s 
  Using cached Cython-0.29.23-cp38-cp38-macosx_10_9_x86_64.whl (1.9 MB)
  Created wheel for sparse-dot-topn: filename=sparse_dot_topn-0.2.9-cp38-cp38-macosx_10_9_x86_64.whl size=57465 sha256=ad02a2270628e171f259064d060821cebff41082d682130709ed6d38f8f184d9
  Stored in directory: /Users/guillaume/Library/Caches/pip/wheels/4a/64/25/2e1cedbf7513fecda8d396a0431f3a1e217f45f1ab565e8ce8
Successfully built sparse-dot-topn


In [7]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
import pandas as pd
import numpy as np
import os
import pickle #optional - for saving outputs
import re
from tqdm import tqdm # used for progress bars (optional)
!pip install ftfy #  text cleaning for decode issues..
import time


import re

from ftfy import fix_text
#transforms company names with assumptions taken from: http://www.legislation.gov.uk/uksi/2015/17/regulation/2/made
def ngrams(string, n=3):
    """Takes an input string, cleans it and converts to ngrams. 
    This script is focussed on cleaning UK company names but can be made generic by removing lines below"""
    string = str(string)
    string = string.lower() # lower case
    string = fix_text(string) # fix text
    string = string.split('t/a')[0] # split on 'trading as' and return first name only
    #string = string.split('trading as')[0] # split on 'trading as' and return first name only
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    chars_to_remove = [")","(",".","|","[","]","{","}","'","-"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']' #remove punc, brackets etc...
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    #string = string.replace('limited', 'ltd')
    #string = string.replace('public limited company', 'plc')
    #string = string.replace('united kingdom', 'uk')
    #string = string.replace('community interest company', 'cic')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

     |████████████████████████████████| 64 kB 185 kB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41913 sha256=b353a03643f2d76350c407a8c800a7ce39e4898da0a5a4e0873820b632d3e1ec
  Stored in directory: /Users/guillaume/Library/Caches/pip/wheels/7f/40/63/4bf603cec3ecc4a26985405834cb47eb8368bfa59e15dde046
Successfully built ftfy


In [34]:
#output data
#df = pd.read_csv('./data/training_data.csv',error_bad_lines=False,encoding='cp1252',sep="\t")
df_x = pd.read_csv('./data/input_x.csv',error_bad_lines=False,encoding='utf-8',sep="\t").iloc[1:,:]
df_y = pd.read_csv('./data/input_y.csv',error_bad_lines=False,encoding='ISO 8859-1',sep="\t",header=None)
df_x

,id,author
1,1,Charles Sykes
2,2,Yogi Berra
3,3,Michael Crichton
4,4,Hegar
5,5,Ken Bensinger
...,...,...
96677,96677,Gavell
96678,96678,Adriana Trigiani
96679,96679,Kien Nguyen
96680,96680,Kyle Mills


In [13]:
artist_name = list(df_x['author'].dropna().unique())
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(artist_name)
tf_idf_matrix

<50745x11912 sparse matrix of type '<class 'numpy.float64'>'
	with 672201 stored elements in Compressed Sparse Row format>

In [15]:
messy_artist = list(df_y[1].unique())
messy_tf_idf_matrix = vectorizer.transform(messy_artist)

In [16]:
!pip install nmslib
import nmslib
from scipy.sparse import csr_matrix # may not be required 
from scipy.sparse import rand # may not be required


# create a random matrix to index
data_matrix = tf_idf_matrix#[0:1000000]

# Set index parameters
# These are the most important ones
M = 80
efC = 1000

num_threads = 4 # adjust for the number of threads
# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='simple_invindx', space='negdotprod_sparse_fast', data_type=nmslib.DataType.SPARSE_VECTOR) 

index.addDataPointBatch(data_matrix)
# Create an index
start = time.time()
index.createIndex() 
end = time.time() 
print('Indexing time = %f' % (end-start))

     |████████████████████████████████| 935 kB 439 kB/s 
  Using cached psutil-5.8.0-cp38-cp38-macosx_10_9_x86_64.whl (236 kB)
     |████████████████████████████████| 188 kB 313 kB/s 
Indexing time = 0.048763


In [17]:
# Number of neighbors 
num_threads = 4
K=1
query_matrix = messy_tf_idf_matrix
start = time.time() 
query_qty = query_matrix.shape[0]
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
end = time.time() 
print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
      (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))

kNN time total=2.185614 (sec), per query=0.000301 (sec), per query adjusted for thread number=0.001204 (sec)


In [19]:
mts =[]
for i in range(len(nbrs)):
  origional_nm = messy_artist[i]
  try:
    matched_nm   = artist_name[nbrs[i][0][0]]
    conf         = nbrs[i][1][0]
  except:
    matched_nm   = "no match found"
    conf         = None
  mts.append([origional_nm,matched_nm,conf])

mts = pd.DataFrame(mts,columns=['origional_name','matched_name','conf'])
#results = df_CF.merge(mts,left_on='buyer',right_on='origional_name')
mts

,origional_name,matched_name,conf
0,Allen Eskens,Allen Eskens,-1.000000
1,C.D. Reiss,C.D. Rose,-0.668106
2,Catherine Gayle,Catherine,-0.704948
3,Daniel J. Siegel,Daniel J. Siegel,-1.000000
4,Debbie Macomber,Debbie Macomber,-1.000000
...,...,...,...
7254,Zondervan,Zondervan,-1.000000
7255,Zora Neale Hurston,Zora Neale Hurston,-1.000000
7256,edited by Brian Kram,Brian Kreb,-0.326442
7257,âConsumer Reportsâ,Bleacher Report,-0.472838


In [21]:
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn #uncomment to install
import sparse_dot_topn.sparse_dot_topn as ct


def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

artist_names = df_y[1].dropna().unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(artist_names)

matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.85)

In [27]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [29]:

matches_df = get_matches_df(matches, artist_name, top=1000)
matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)
matches_df.sort_values(['similairity'], ascending=False).head(20)

,left_side,right_side,similairity
256,Christie Craig,David Clarke,0.937906
258,David Clarke,Christie Craig,0.937906
875,Elizabeth Swados,Lecia Cornwall,0.927152
870,Lecia Cornwall,Elizabeth Swados,0.927152
657,Maria Goldverg,Andrea Schulz,0.920448
14,Faith Hunter,Rod Duncan,0.911747
505,Ally Condie,Mussolini,0.910818
507,Mussolini,Ally Condie,0.910818
181,Chloe Caldwell,Brain Fitness Lab Lisa Mosconi,0.905992
183,Brain Fitness Lab Lisa Mosconi,Chloe Caldwell,0.905992


In [37]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re



artist_name_clean = df_x['author'].dropna().unique()

print('Vectorizing the data - this could take a few minutes for large datasets...')
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(artist_name_clean)
print('Vectorizing completed...')

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)

org_column = 'buyer' #column to match against in the messy data
unique_artist = set(df_y['author'].values) # set used for increased performance


###matching query:
def getNearestN(query):
  queryTFIDF_ = vectorizer.transform(query)
  distances, indices = nbrs.kneighbors(queryTFIDF_)
  return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)

unique_org = list(unique_org) #need to convert back to a list
print('finding matches...')
matches = []
for i,j in enumerate(indices):
  temp = [round(distances[i][0],2), artist_name_clean.values[j][0][0],unique_artist[i]]
  matches.append(temp)

print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Origional name'])
print('Done') 

Vectorizing the data - this could take a few minutes for large datasets...
Vectorizing completed...
getting nearest n...


KeyboardInterrupt: 

In [35]:
df_y = df_y.rename(columns={1: "author"})
df_y

,0,author
0,1,Allen Eskens
1,2,C.D. Reiss
2,3,Catherine Gayle
3,4,Daniel J. Siegel
4,5,Debbie Macomber
...,...,...
8297,8298,Zondervan
8298,8299,Zora Neale Hurston
8299,8300,edited by Brian Kram
8300,8301,âConsumer Reportsâ


# Last try Guillaume
* https://towardsdatascience.com/fuzzy-string-match-with-python-on-large-dataset-and-why-you-should-not-use-fuzzywuzzy-4ec9f0defcd

In [1]:
import numpy as np
import pandas as pd
import random
import gc
import timeit
import sys
from datetime import datetime as dt
import re

import matplotlib.pyplot as plt

In [2]:
df_x = pd.read_csv('./data/input_x.csv',error_bad_lines=False,encoding='utf-8',sep="\t").iloc[1:,:]
df_y = pd.read_csv('./data/input_y.csv',error_bad_lines=False,encoding='ISO 8859-1',sep="\t",header=None)

df_x = df_x.iloc[:1000,:]
df_y = df_y.iloc[:1000,:]

In [3]:
df_y.head(5)

,0,1
0,1,Allen Eskens
1,2,C.D. Reiss
2,3,Catherine Gayle
3,4,Daniel J. Siegel
4,5,Debbie Macomber


In [4]:
def text_prepare(text):
    """
        text: a string
        
        return: a clean string
    """
    REPLACE_BY_SPACE_RE = re.compile('[\n\"\'/(){}\[\]\|@,;#]')
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.lower()

    # delete stopwords from text
    text = ' '.join([word for word in text.split()]) 
    text = text.strip()
    return text

In [5]:
df_x['author'] = df_x['author'].astype('string')
df_x.dtypes

id         int64
author    string
dtype: object

In [6]:
df_y.head(10)

,0,1
0,1,Allen Eskens
1,2,C.D. Reiss
2,3,Catherine Gayle
3,4,Daniel J. Siegel
4,5,Debbie Macomber
5,6,Dick Morris
6,7,Dolen Perkins-Valdez
7,8,Donna Woolfolk Cross
8,9,Douglas Preston and Lincoln Child
9,10,Emily Giffin


In [7]:
df_x = df_x.fillna("none")

In [8]:
df_x['Author'] = df_x['author'].apply(lambda x: text_prepare(x))
df_y['author'] = df_y[1].apply(lambda x: text_prepare(x))
df_x.head(5)

,id,author,Author
1,1,Charles Sykes,charles sykes
2,2,Yogi Berra,yogi berra
3,3,Michael Crichton,michael crichton
4,4,Hegar,hegar
5,5,Ken Bensinger,ken bensinger


In [9]:
#transform text to vectors with TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5, token_pattern='(\S+)')
tf_idf_matrix_x = tfidf_vectorizer.fit_transform(df_x['Author'])
tf_idf_matrix_y = tfidf_vectorizer.fit_transform(df_y['author'])

In [10]:
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))


import time
t1 = time.time()

# adjust lower bound: 0.8
# keep top 10 similar results
matches = awesome_cossim_top(tf_idf_matrix_x, tf_idf_matrix_y.transpose(), 10, 0.8)

t = time.time()-t1
print("finished in:", t)

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
finished in: 0.002249002456665039


In [11]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'TITLE': left_side,
                          'SIMILAR_TITLE': right_side,
                           'similairity_score': similairity})
  
  
  
matches_df = pd.DataFrame()
matches_df = get_matches_df(matches, df_x['Author'], top=10000)
# Remove all exact matches
matches_df = matches_df[matches_df['similairity_score'] < 0.99999] 
matches_df.sample(10)

KeyError: 0

In [52]:
matches_df.sort_values(["similairity_score"], ascending=False).head(20)

,TITLE,SIMILAR_TITLE,similairity_score
9708,nebula awards,nebula awards awards,0.952539
3658,arthur ellis unhanged arthur award,arthur ellis award,0.947048
3657,arthur ellis unhanged arthur award,arthur ellis award,0.947048
3656,arthur ellis unhanged arthur award,arthur ellis award,0.947048
7281,nino ricci,award nino ricci,0.944778
1949,charlotte zolotow,charlotte zolotow award,0.933302
8574,tom o donnell,sunshine o donnell,0.921246
9507,wallace stegner fellow abigail ulman,wallace stegner fellow,0.919925
9508,wallace stegner fellow abigail ulman,wallace stegner fellow,0.919925
9509,wallace stegner fellow abigail ulman,wallace stegner fellow,0.919925
